In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
import numpy as np
import scipy as sp
import pandas as pd
import theano as th
import theano.tensor as tt
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sb
import g3py as g3
import pymc3 as pm

g3.style_seaborn()
#g3.style_normal()
#g3.style_big()
#g3.style_big_seaborn()
g3.style_widget()


th.config.NanGuardMode.big_is_error = False
#th.config.mode = 'DebugMode'
#th.config.mode = 'FAST_RUN'
th.config.exception_verbosity = 'high'

# Data

In [2]:
#x, y = g3.load_sunspots()
x, y = g3.load_co2()
x = x-x[0]

+-------+-------------------+
| co2   | R Documentation   |
+-------+-------------------+

Mauna Loa Atmospheric CO2 Concentration
---------------------------------------

Description
~~~~~~~~~~~

Atmospheric concentrations of CO\ *2* are expressed in parts per million
(ppm) and reported in the preliminary 1997 SIO manometric mole fraction
scale.

Usage
~~~~~

::

    co2

Format
~~~~~~

A time series of 468 observations; monthly from 1959 to 1997.

Details
~~~~~~~

The values for February, March and April of 1964 were missing and have
been obtained by interpolating linearly between the values for January
and May of 1964.

Source
~~~~~~

Keeling, C. D. and Whorf, T. P., Scripps Institution of Oceanography
(SIO), University of California, La Jolla, California USA 92093-0220.

`ftp://cdiac.esd.ornl.gov/pub/maunaloa-co2/maunaloa.co2 <ftp://cdiac.esd.ornl.gov/pub/maunaloa-co2/maunaloa.co2>`__.

References
~~~~~~~~~~

Cleveland, W. S. (1993) *Visualizing Data*. New Jersey: Summit Press.

Ex

In [3]:
obs_j, x_obs, y_obs, test_j, x_test, y_test = g3.random_obs(x, y, 0.5, 0.5)

Total: 468 | Obs: 117 (25.0%)


In [4]:
with g3.Model() as model:
    mean = g3.Bias()
    ker = g3.SE(x)
    trans = g3.Identity(y_obs) @ g3.LogShifted(y_obs)
    tgp = g3.TGP(x, mean, ker, trans, noise=True, hidden=y)
    tgp.describe(str(k),'SUNACTIVITY','YEAR')
    tgp.observed(x_obs,y_obs)
    tgp.testing(x_test,y_test)
    tgp.compile()
#gp.plot_gp(gp.find_default(), samples=10)

NameError: name 'gp' is not defined

In [ ]:
trans.hypers

In [ ]:
_ = plt.plot(gp.compiles['covariance'](**gp.get_params())[0])
_ = plt.plot(gp.compiles['covariance'](**gp.get_params())[len(x)//2])
_ = plt.plot(gp.compiles['covariance'](**gp.get_params())[-1])

In [ ]:
points = gp.find_MAP(points=6, plot=True, samples=10)

In [ ]:
with g3.Model() as model:
    m = g3.Bias()
    k = g3.PER(x)# + g3.SE(x)
    gp = g3.GP(x, m, k, noise=True, hidden=y)
    gp.describe(str(k),'SUNACTIVITY','YEAR')
    gp.observed(x_obs,y_obs)
    gp.testing(x_test,y_test)
    gp.compile()
gp.plot_gp(gp.find_default(), samples=3)

In [ ]:
gp.widget_params()

In [ ]:
points = gp.find_MAP(start=gp.get_params(),points=6, plot=True, samples=3)

In [ ]:
with g3.Model() as model:
    m = g3.Bias()
    k = g3.SM(x)
    gp = g3.GP(x, m, k, noise=True, hidden=y)
    gp.describe(str(k),'SUNACTIVITY','YEAR')
    gp.observed(x_obs,y_obs)
    gp.testing(x_test,y_test)
    gp.compile()
gp.plot_gp(gp.find_default(), samples=3)

In [ ]:
gp.widget_params()

In [ ]:
points = gp.find_MAP(start=gp.get_params(),points=6, plot=True, samples=3)

In [ ]:
with g3.Model() as model:
    m = g3.Bias()
    k = g3.SM(x, name='SM1')+g3.SM(x, name='SM2')
    gp = g3.GP(x, m, k, noise=True, hidden=y)
    gp.describe(str(k),'SUNACTIVITY','YEAR')
    gp.observed(x_obs,y_obs)
    gp.testing(x_test,y_test)
    gp.compile()
gp.plot_gp(gp.find_default(), samples=3)

In [ ]:
gp.widget_params()

In [ ]:
points = gp.find_MAP(start=gp.get_params(),points=6, plot=True, samples=3)

In [ ]:
name, logp, params = points[-3]

In [ ]:
traces = gp.sample_hypers(params, samples=10000,advi=False)

In [ ]:
g3.save_trace(traces)

In [ ]:
traces.varnames.clear()
for v in gp.model.vars:
    traces.varnames.append(v.name)
traces.varnames

In [ ]:
g3.style_seaborn()
g3.traceplot(traces, plot_transformed=True)

In [ ]:
datatraces = g3.datatrace(model, traces)
g3.save_datatrace(datatraces)
datatraces.describe().T

In [ ]:
items_ll = ['niter','ll']
items_k1 = ['GP_SM1_M','GP_SM1_S_log_','GP_SM1_Var_log_']
items_k2 = ['GP_SM3_M','GP_SM2_S_log_','GP_SM2_Var_log_']
items_m = ['GP_Bias_Constant','GP_Noise_Var_log_']

g3.plot_datatrace(datatraces,items_ll+items_k1)
g3.plot_datatrace(datatraces,items_ll+items_k2)
g3.plot_datatrace(datatraces,items_ll+items_m)


g3.plot_datatrace(datatraces,items_k1+items_k2)
g3.plot_datatrace(datatraces,items_k1+items_m)
g3.plot_datatrace(datatraces,items_k2+items_m)




In [ ]:
gp.widget_trace(traces)

In [ ]:
tgp.plot_tgp(tgp.get_params(), samples=10)